In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from random import randrange
import threading
import time
import numpy as np
import pandas as pd
from datetime import datetime
import serial
import tkinter as tk
import time
import ipywidgets as widgets

In [3]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from random import randrange
import threading
import time
import numpy as np
import pandas as pd
from datetime import datetime
import serial
import serial.tools.list_ports
import traceback
import tkinter as tk
import time
import ipywidgets as widgets
import os
# Class for live update of a graph
class LiveGraph:
    def __init__(self, port_name = '/dev/cu.usbmodem145301', filename = None):
        # Setting csv file name
        self.filename = filename
        
        # Setting the data
        columns = ["Real_time", "Serial_time", "Current_temp", "Target_temp", "Heating_output", "Temp_diff", "Is_Heating"]
        self.df = pd.DataFrame(columns = columns)
        self.maxRows = 6000
        
        # Setting up the button
        self.button = widgets.Button(description="STOP!") 
        self.output = widgets.Output()
        self.break_cycle = False
        self.button.on_click(self.on_button_clicked)

        # Setting input 
        self.text_input = widgets.Text(description="Send to Arduino:")
        self.text_input.on_submit(self.on_text_submit)
        
        # Setting up plot
        self.figure, self.axes = plt.subplots(3)
        self.lines = [] # Lines for the data
        self.texts = [] # Text plot
        color = ['b', 'r', 'b', 'b']
        self.coli = [ "Target_temp", "Current_temp", "Temp_diff", "Heating_output", "Is_Heating"]
        for i, n, c in zip([0, 0, 1, 2], self.coli, color):
            line, = self.axes[i].plot_date([],[], c,label = n, ls = '-', marker='None')
            self.lines.append(line)
            text = self.axes[i].text(0,0,' ', color = c)
            self.texts.append(text)
        for ax in self.axes: ax.legend(loc="upper left")
            
        # Setting up serial port
        self.port = port_name
        self.ser = serial.Serial(self.port, 9600, timeout=1.3)
        time.sleep(2)
        self.ser.flush()
        
        # Start animation
        self.animation = FuncAnimation(self.figure, self.update, interval=1000, cache_frame_data=False)

        # Start threading
        self.th = threading.Thread(target=self.thread_f, daemon=True)
        self.th.start()

    
    def update(self, frame):
        for (li, te, col) in zip( self.lines, self.texts, self.coli):
            li.set_data(self.df['Real_time'],self.df[col])
            te.set_position((self.df['Real_time'].iloc[-1],self.df[col].iloc[-1]))
            te.set_text(round(self.df[col].iloc[-1],2))
            
        for ax in self.axes:
            ax.relim()
            ax.autoscale_view()
            
        if self.break_cycle:
            self.animation.event_source.stop()
            
        return self.lines

    
    def on_button_clicked(self, event):
        print("Button pressed: break_cycle:", self.break_cycle)
        self.break_cycle = True
        self.ser.close()
        
    def on_text_submit(self, text_widget):
        value = text_widget.value
        print(f"Sending to Arduino: {value}")
        self.ser.write((value + '\n').encode())  # Write to serial port
        text_widget.value = ""  # Clear input after sending
    
    def show(self):
        plt.show()
        display(self.button, self.text_input, self.output)


    def thread_f(self):
        while not self.break_cycle:
            # read the serial port 
            line = self.ser.readline().decode().strip()
            nums = line.split()
            if len(nums) == 5:
                try:
                    values = [float(num) for num in nums[0:4]]
                    temp_df = pd.DataFrame({"Real_time":pd.Timestamp.now(), 
                                    "Serial_time":[values[0]/1000], 
                                    "Current_temp":[values[1]], 
                                    "Target_temp":[values[2]], 
                                    "Heating_output":[values[3]],
                                    "Temp_diff":[values[1]-values[2]],
                                    "Is_Heating": [nums[4]]})
                    # Append your result into the dataframe
                    self.df = pd.concat([self.df, temp_df], ignore_index=True)
                    if self.filename is not None:
                        temp_df.to_csv(self.filename, mode='a', header=not os.path.exists(self.filename))
                    if len(self.df) > self.maxRows:
                        self.df = self.df.tail(self.maxRows)
                except Exception as err:
                    print(f"Unexpected {err=}, {type(err)=}")
                    print(f"The serial out put is {line=}")

            time.sleep(0.001)


IndentationError: expected an indented block (2659044587.py, line 103)

# Running the program


In [6]:
g = LiveGraph(filename = "2025_8_15.csv")
g.show()

NameError: name 'LiveGraph' is not defined

# Save your results!

In [130]:
g.df.to_csv(r'C:\Users\Wiggi\Desktop\James\temp_data\1_10_2025.csv')

In [131]:
%pip install pyserial pandas numpy matplotlib ipywidgets


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [132]:
# Get the list of available serial ports
ports = serial.tools.list_ports.comports()

# Extract and display port device names
port_list = [port.device for port in ports]
print(port_list)

['/dev/cu.BLTH', '/dev/cu.URT1', '/dev/cu.URT2', '/dev/cu.Bluetooth-Incoming-Port', '/dev/cu.usbmodem145301']


In [133]:
serial.Serial('/dev/cu.usbmodem145301', 9600, timeout=1.3)

SerialException: [Errno 16] could not open port /dev/cu.usbmodem145301: [Errno 16] Resource busy: '/dev/cu.usbmodem145301'